In [10]:
import pandas as pd
from numpy import sqrt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [4]:
results = pd.read_csv('../../Data/ResultData/ADAUSDT_7_5_wave_trend_results.csv')
df = results[results['clean_gains'] != 0]
df = df.drop(columns = ['symbol', 'indicator', 'start_date', 'end_date', 'mean', 'n_total_partitions', 'n_partitions'])

In [5]:
df.head()

,ob_level,os_level,k,std,skewness,kurtosis,entropy,clean_gains
0,43,-63,0.001,0.002346,0.055612,-0.985233,9.217999,0.157059
1,43,-63,0.001,0.002326,0.409831,-0.210873,9.218010,4.233419
2,43,-63,0.002,0.005681,0.283234,-1.222078,9.216114,1.832861
3,43,-63,0.001,0.005335,-0.429036,-1.028970,9.216932,-13.175106
4,43,-63,0.002,0.002326,0.409831,-0.210873,9.218010,3.616770


In [6]:
X, y = df.values[:, :-1], df.values[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Scikit.learn MLP regressor

In [15]:
mlp = MLPRegressor(random_state=1, max_iter=1000)

mlp.fit(X_train, y_train)

mlp_y_pred = mlp.predict(X_test[:2])

In [16]:
mlp_r2 = metrics.r2_score(y_test, mlp_y_pred)
print(mlp_r2)

ValueError: Found input variables with inconsistent numbers of samples: [958, 2]

# Tensor Flow

In [10]:
# determine the number of input features
n_features = X_train.shape[1]

# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mse')

# fit the model
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)

In [11]:
# evaluate the model
error = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (error, sqrt(error)))
# make a prediction
row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
yhat = model.predict([row])
print('Predicted: %.3f' % yhat)

MSE: 26.755, RMSE: 5.172


ValueError: in user code:

    E:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    E:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    E:\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    E:\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    E:\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    E:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1445 run_step  **
        outputs = model.predict_step(data)
    E:\Anaconda\lib\site-packag